In [1]:
import pandas as pd
import numpy as np
import requests
import re
import datetime


In [2]:
df1 = pd.read_csv('Data\\wallstreetbets_submissions_zstextract.csv')
df2 = pd.read_csv('Data\\wallstreetbets_posts_2023.csv')

C:\Users\Administrator\AppData\Local\Temp\ipykernel_15220\1361390096.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('Data\\wallstreetbets_submissions_zstextract.csv')


In [3]:
df2['type'] = 'Post'
df2['name'] = str(np.nan)

df2 = df2.rename(columns={'selftext' : 'body'})

df1['upvote_ratio'] = [round(up/(up +down)) if (up+down) != 0 else up for up, down in zip(df1['upvotes'], df1['downvotes'])]

df1 = df1[['type', 'id', 'title', 'body', 'upvote_ratio', 'author', 'name',
       'created_utc', 'url', 'score', 'num_comments', 'permalink']]
df2 = df2[['type', 'id', 'title', 'body', 'upvote_ratio', 'author', 'name',
       'created_utc', 'url', 'score', 'num_comments', 'permalink']]

df = pd.concat([df1, df2])

# Converting timestamp to date time format
df['created_timestamp'] = df['created_utc'].apply(datetime.datetime.fromtimestamp)

# Filtering for data from 2022 onwards
df = df[(df['created_timestamp'] >= '2022-01-01')]

df.head()

,type,id,title,body,upvote_ratio,author,name,created_utc,url,score,num_comments,permalink,created_timestamp
1973933,Post,rtcytw,Possible reasons you aren't 3D printing Tendy ...,[removed],0.0,[deleted],t3_rtcytw,1641016911,https://www.reddit.com/r/wallstreetbets/commen...,1,2,/r/wallstreetbets/comments/rtcytw/possible_rea...,2022-01-01 00:01:51
1973934,Post,rtczjg,[deleted by user],[removed],0.0,[deleted],t3_rtczjg,1641016983,https://www.reddit.com/r/wallstreetbets/commen...,1,1,/r/wallstreetbets/comments/rtczjg/deleted_by_u...,2022-01-01 00:03:03
1973935,Post,rtd4ey,Happy New Years you retards❤️,NaN,0.0,carsonthecarsinogen,t3_rtd4ey,1641017504,https://i.redd.it/6ch4n3l1q0981.jpg,1,1,/r/wallstreetbets/comments/rtd4ey/happy_new_ye...,2022-01-01 00:11:44
1973936,Post,rtd7ym,[deleted by user],[removed],0.0,[deleted],t3_rtd7ym,1641017906,https://www.reddit.com/r/wallstreetbets/commen...,1,0,/r/wallstreetbets/comments/rtd7ym/deleted_by_u...,2022-01-01 00:18:26
1973937,Post,rtdg3p,I DIDN'T HEAR NO BELL! 🔔,[deleted],0.0,[deleted],t3_rtdg3p,1641018757,https://i.redd.it/4632sqort0981.jpg,166,34,/r/wallstreetbets/comments/rtdg3p/i_didnt_hear...,2022-01-01 00:32:37


In [4]:
data = df
data.shape

(271513, 13)

In [5]:
data.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 271513 entries, 1973933 to 27202
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   type               271513 non-null  object        
 1   id                 271513 non-null  object        
 2   title              271510 non-null  object        
 3   body               163647 non-null  object        
 4   upvote_ratio       271513 non-null  float64       
 5   author             271513 non-null  object        
 6   name               271513 non-null  object        
 7   created_utc        271513 non-null  int64         
 8   url                238627 non-null  object        
 9   score              271513 non-null  int64         
 10  num_comments       271513 non-null  int64         
 11  permalink          271513 non-null  object        
 12  created_timestamp  271513 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64

Checking if the four rows which are null for column 'title' have any information

In [6]:
data[data['title'].isnull()]

,type,id,title,body,upvote_ratio,author,name,created_utc,url,score,num_comments,permalink,created_timestamp
2035488,Post,tyn6so,NaN,NaN,0.0,arh910,t3_tyn6so,1649366505,https://airdrop.unlockd.finance?kid=23Z40X,1,2,/r/wallstreetbets/comments/tyn6so/null/,2022-04-07 16:21:45
2109284,Post,wm8t2v,NaN,NaN,0.0,GroundbreakingBad426,t3_wm8t2v,1660269388,https://win.grizzly.fi/?kid=2DSC1Q,0,3,/r/wallstreetbets/comments/wm8t2v/null/,2022-08-11 20:56:28
2133963,Post,wvixic,NaN,NaN,0.0,ko308ko,t3_wvixic,1661242263,https://airdrop.unlockd.finance?kid=2EC28F,1,2,/r/wallstreetbets/comments/wvixic/null/,2022-08-23 03:11:03


The 4 rows have no information and since it's only 4 rows, dropping those rows where the title & body are null

In [7]:
data = data.dropna(subset=['title', 'body'], how='all')
data.shape

(271510, 13)

1. Dropping the column 'name' as it is 86% null and is not needed as the author is already available with no null values
2. Dropping the column 'url as it is 98% null and the same information is given by permalink
3. Dropping column 'num_comments' as it is not the latest updated
4. Keeping the column 'permalink' for checking against Reddit portal before modeling

In [8]:
data = data.drop(['name', 'url', 'num_comments'], axis=1)
data.shape

(271510, 10)

In [9]:
data.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 271510 entries, 1973933 to 27202
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   type               271510 non-null  object        
 1   id                 271510 non-null  object        
 2   title              271510 non-null  object        
 3   body               163647 non-null  object        
 4   upvote_ratio       271510 non-null  float64       
 5   author             271510 non-null  object        
 6   created_utc        271510 non-null  int64         
 7   score              271510 non-null  int64         
 8   permalink          271510 non-null  object        
 9   created_timestamp  271510 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2), object(6)
memory usage: 22.8+ MB


In [10]:
data = data.drop_duplicates(subset=['id'], keep='first')

Removing links, emojis and special characters to make the data cleaner

In [11]:
data_cleaned = data
data_cleaned['body'] = data_cleaned['body'].map(lambda x: re.sub('[^a-zA-Z0-9 @ . , : - _]', '', str(x)))
data_cleaned['title'] = data_cleaned['title'].map(lambda x: re.sub('[^a-zA-Z0-9 @ . , : - _]', '', str(x)))
data_cleaned.head()

,type,id,title,body,upvote_ratio,author,created_utc,score,permalink,created_timestamp
1973933,Post,rtcytw,Possible reasons you arent 3D printing Tendy T...,removed,0.0,[deleted],1641016911,1,/r/wallstreetbets/comments/rtcytw/possible_rea...,2022-01-01 00:01:51
1973934,Post,rtczjg,deleted by user,removed,0.0,[deleted],1641016983,1,/r/wallstreetbets/comments/rtczjg/deleted_by_u...,2022-01-01 00:03:03
1973935,Post,rtd4ey,Happy New Years you retards,nan,0.0,carsonthecarsinogen,1641017504,1,/r/wallstreetbets/comments/rtd4ey/happy_new_ye...,2022-01-01 00:11:44
1973936,Post,rtd7ym,deleted by user,removed,0.0,[deleted],1641017906,1,/r/wallstreetbets/comments/rtd7ym/deleted_by_u...,2022-01-01 00:18:26
1973937,Post,rtdg3p,I DIDNT HEAR NO BELL,deleted,0.0,[deleted],1641018757,166,/r/wallstreetbets/comments/rtdg3p/i_didnt_hear...,2022-01-01 00:32:37


In [12]:
data_cleaned['title_body'] = data_cleaned['title'] + ' ' + data_cleaned['body']
data_cleaned = data_cleaned.drop(['body'], axis=1)
data_cleaned.head()

,type,id,title,upvote_ratio,author,created_utc,score,permalink,created_timestamp,title_body
1973933,Post,rtcytw,Possible reasons you arent 3D printing Tendy T...,0.0,[deleted],1641016911,1,/r/wallstreetbets/comments/rtcytw/possible_rea...,2022-01-01 00:01:51,Possible reasons you arent 3D printing Tendy T...
1973934,Post,rtczjg,deleted by user,0.0,[deleted],1641016983,1,/r/wallstreetbets/comments/rtczjg/deleted_by_u...,2022-01-01 00:03:03,deleted by user removed
1973935,Post,rtd4ey,Happy New Years you retards,0.0,carsonthecarsinogen,1641017504,1,/r/wallstreetbets/comments/rtd4ey/happy_new_ye...,2022-01-01 00:11:44,Happy New Years you retards nan
1973936,Post,rtd7ym,deleted by user,0.0,[deleted],1641017906,1,/r/wallstreetbets/comments/rtd7ym/deleted_by_u...,2022-01-01 00:18:26,deleted by user removed
1973937,Post,rtdg3p,I DIDNT HEAR NO BELL,0.0,[deleted],1641018757,166,/r/wallstreetbets/comments/rtdg3p/i_didnt_hear...,2022-01-01 00:32:37,I DIDNT HEAR NO BELL deleted


In [13]:
data_cleaned.shape

(250300, 10)

Processing comments for the posts

In [14]:
comments = pd.read_csv('Data\\comments_combined.csv')
comments.tail()

,author,body,comment_type,created_utc,id,link_id,parent_id,permalink,score
441995,scothu,This guy earning reports,NaN,1678011738,jazwk97,t3_11iki8f,4.202298e+10,/r/wallstreetbets/comments/11iki8f/lmao_elon_r...,1
441996,VisualMod,I am a bot from /r/wallstreetbets. Your submis...,NaN,1678011738,jazwk9n,t3_11its7h,NaN,/r/wallstreetbets/comments/11its7h/chris_rock_...,1
441997,AutoModerator,Your post has been removed because a single li...,NaN,1678011729,jazwjvo,t3_11its7h,NaN,/r/wallstreetbets/comments/11its7h/chris_rock_...,1
441998,Dan_inKuwait,Unusual whales posted a graph a while back tha...,NaN,1678011673,jazwhlo,t3_11isu8t,4.202376e+10,/r/wallstreetbets/comments/11isu8t/bounty_hunt...,1
441999,[deleted],[removed],NaN,1678011667,jazwhd1,t3_11hz85i,NaN,/r/wallstreetbets/comments/11hz85i/0dte_option...,1


In [15]:
comments.shape

(442000, 9)

In [16]:
comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442000 entries, 0 to 441999
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   author        442000 non-null  object 
 1   body          442000 non-null  object 
 2   comment_type  0 non-null       float64
 3   created_utc   442000 non-null  int64  
 4   id            442000 non-null  object 
 5   link_id       442000 non-null  object 
 6   parent_id     264110 non-null  float64
 7   permalink     442000 non-null  object 
 8   score         442000 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 30.3+ MB


Drop columns 'comment_type' as it has all nulls and is not required; and 'parent_id' as 'id' is the ID of the comment and 'link_id' gives information about which post it was a comment for

In [17]:
comments = comments.drop(['comment_type', 'parent_id'], axis=1)
comments.shape

(442000, 7)

In [18]:
comments['type'] = 'Comment'

comments['link_id'] = comments['link_id'].str.replace('t3_', '')

comments['body'] = comments['body'].replace('[removed]', np.nan)
comments['body'] = comments['body'].replace('[deleted]', np.nan)
comments.tail()

,author,body,created_utc,id,link_id,permalink,score,type
441995,scothu,This guy earning reports,1678011738,jazwk97,11iki8f,/r/wallstreetbets/comments/11iki8f/lmao_elon_r...,1,Comment
441996,VisualMod,I am a bot from /r/wallstreetbets. Your submis...,1678011738,jazwk9n,11its7h,/r/wallstreetbets/comments/11its7h/chris_rock_...,1,Comment
441997,AutoModerator,Your post has been removed because a single li...,1678011729,jazwjvo,11its7h,/r/wallstreetbets/comments/11its7h/chris_rock_...,1,Comment
441998,Dan_inKuwait,Unusual whales posted a graph a while back tha...,1678011673,jazwhlo,11isu8t,/r/wallstreetbets/comments/11isu8t/bounty_hunt...,1,Comment
441999,[deleted],NaN,1678011667,jazwhd1,11hz85i,/r/wallstreetbets/comments/11hz85i/0dte_option...,1,Comment


Drop all rows where body is null as it provided no information

In [19]:
comments = comments.dropna(subset=['body'])
comments.shape

(410682, 8)

In [20]:
duplicates = comments[comments.duplicated(['id'], keep=False)]
duplicates

,author,body,created_utc,id,link_id,permalink,score,type
0,[deleted],Gambling addiction makes that a tough ask.,1641081598,hqvavaz,rt35kz,/r/wallstreetbets/comments/rt35kz/year_end_dis...,1,Comment
1,PrincPaco,My biggest problem is that I'm a wallflower. I...,1641081592,hqvaut8,rt35kz,/r/wallstreetbets/comments/rt35kz/year_end_dis...,1,Comment
2,fortytree,His dick is like his nose,1641081585,hqvau9a,rt35kz,/r/wallstreetbets/comments/rt35kz/year_end_dis...,2,Comment
3,Melonpimp,Tell her to cuddle your cock in her mouth. So ...,1641081578,hqvatqs,rt35kz,/r/wallstreetbets/comments/rt35kz/year_end_dis...,4,Comment
4,VisualMod,I am a bot from /r/wallstreetbets. ...,1641081578,hqvatqk,rtw8a6,/r/wallstreetbets/comments/rtw8a6/is_arkk_a_go...,1,Comment
...,...,...,...,...,...,...,...,...
441994,arcadeScore,This.,1678011757,jazwl0d,11inr3o,/r/wallstreetbets/comments/11inr3o/nvidia_phuc...,1,Comment
441995,scothu,This guy earning reports,1678011738,jazwk97,11iki8f,/r/wallstreetbets/comments/11iki8f/lmao_elon_r...,1,Comment
441996,VisualMod,I am a bot from /r/wallstreetbets. Your submis...,1678011738,jazwk9n,11its7h,/r/wallstreetbets/comments/11its7h/chris_rock_...,1,Comment
441997,AutoModerator,Your post has been removed because a single li...,1678011729,jazwjvo,11its7h,/r/wallstreetbets/comments/11its7h/chris_rock_...,1,Comment


In [21]:
comments = comments.drop_duplicates(subset=['id'], keep='first')

In [22]:
# Converting timestamp to date time format
comments['created_timestamp'] = comments['created_utc'].apply(datetime.datetime.fromtimestamp)

# Filtering for comments from 2022 onwards
comments = comments[(comments['created_timestamp'] >= '2022-01-01') & (comments['created_timestamp'] < '2023-03-01')]
comments.tail()

,author,body,created_utc,id,link_id,permalink,score,type,created_timestamp
435095,Pickle_Rick_Goat,What strike?,1677628508,jaf4dmu,11ejs3m,/r/wallstreetbets/comments/11ejs3m/what_are_yo...,1,Comment,2023-02-28 17:55:08
435096,leeghandiz4me,"all chinese, japanese, and german prolly",1677628506,jaf4di4,11eem6e,/r/wallstreetbets/comments/11eem6e/us_homebuye...,1,Comment,2023-02-28 17:55:06
435097,ZasdfUnreal,I'll give one optimistic scenario. It's like g...,1677628501,jaf4d28,11e98of,/r/wallstreetbets/comments/11e98of/explaining_...,2,Comment,2023-02-28 17:55:01
435098,akrebo18,Squeeze deez nuggets,1677628496,jaf4clg,11ejs3m,/r/wallstreetbets/comments/11ejs3m/what_are_yo...,3,Comment,2023-02-28 17:54:56
435099,31andnotdone,oh,1677628494,jaf4ci8,11ejs3m,/r/wallstreetbets/comments/11ejs3m/what_are_yo...,7,Comment,2023-02-28 17:54:54


Removing links, emojis and special characters to make the data cleaner

In [23]:
comments_cleaned = comments
comments_cleaned['body'] = comments_cleaned['body'].map(lambda x: re.sub('[^a-zA-Z0-9 @ . , : - _]', '', str(x)))
comments_cleaned.head()

,author,body,created_utc,id,link_id,permalink,score,type,created_timestamp
0,[deleted],Gambling addiction makes that a tough ask.,1641081598,hqvavaz,rt35kz,/r/wallstreetbets/comments/rt35kz/year_end_dis...,1,Comment,2022-01-01 17:59:58
1,PrincPaco,My biggest problem is that Im a wallflower. I ...,1641081592,hqvaut8,rt35kz,/r/wallstreetbets/comments/rt35kz/year_end_dis...,1,Comment,2022-01-01 17:59:52
2,fortytree,His dick is like his nose,1641081585,hqvau9a,rt35kz,/r/wallstreetbets/comments/rt35kz/year_end_dis...,2,Comment,2022-01-01 17:59:45
3,Melonpimp,Tell her to cuddle your cock in her mouth. So ...,1641081578,hqvatqs,rt35kz,/r/wallstreetbets/comments/rt35kz/year_end_dis...,4,Comment,2022-01-01 17:59:38
4,VisualMod,I am a bot from rwallstreetbets. ...,1641081578,hqvatqk,rtw8a6,/r/wallstreetbets/comments/rtw8a6/is_arkk_a_go...,1,Comment,2022-01-01 17:59:38


In [24]:
comments_cleaned = comments_cleaned[~((comments_cleaned['author'] == 'VisualMod') & (comments_cleaned['body'].str.contains('User Report')))]
comments_cleaned = comments_cleaned[~((comments_cleaned['author'] == 'VisualMod') & (comments_cleaned['body'].str.contains('I am a bot')))]
comments_cleaned.tail()

,author,body,created_utc,id,link_id,permalink,score,type,created_timestamp
435095,Pickle_Rick_Goat,What strike,1677628508,jaf4dmu,11ejs3m,/r/wallstreetbets/comments/11ejs3m/what_are_yo...,1,Comment,2023-02-28 17:55:08
435096,leeghandiz4me,"all chinese, japanese, and german prolly",1677628506,jaf4di4,11eem6e,/r/wallstreetbets/comments/11eem6e/us_homebuye...,1,Comment,2023-02-28 17:55:06
435097,ZasdfUnreal,Ill give one optimistic scenario. Its like giv...,1677628501,jaf4d28,11e98of,/r/wallstreetbets/comments/11e98of/explaining_...,2,Comment,2023-02-28 17:55:01
435098,akrebo18,Squeeze deez nuggets,1677628496,jaf4clg,11ejs3m,/r/wallstreetbets/comments/11ejs3m/what_are_yo...,3,Comment,2023-02-28 17:54:56
435099,31andnotdone,oh,1677628494,jaf4ci8,11ejs3m,/r/wallstreetbets/comments/11ejs3m/what_are_yo...,7,Comment,2023-02-28 17:54:54


In [25]:
comments_cleaned.shape

(39055, 9)

In [26]:
comments_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39055 entries, 0 to 435099
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   author             39055 non-null  object        
 1   body               39055 non-null  object        
 2   created_utc        39055 non-null  int64         
 3   id                 39055 non-null  object        
 4   link_id            39055 non-null  object        
 5   permalink          39055 non-null  object        
 6   score              39055 non-null  int64         
 7   type               39055 non-null  object        
 8   created_timestamp  39055 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(6)
memory usage: 3.0+ MB


Create a combined comments and post DataFrame

In [27]:
# jaf4dmu
comments_cleaned[comments_cleaned['link_id'] == '11ejs3m']

,author,body,created_utc,id,link_id,permalink,score,type,created_timestamp
435000,BeastSmitty,Gottamm you aint kiddin,1677628797,jaf5181,11ejs3m,/r/wallstreetbets/comments/11ejs3m/what_are_yo...,1,Comment,2023-02-28 17:59:57
435001,CuckLordTren,Solid 4 You act as if a 4 can be anything but ...,1677628795,jaf5102,11ejs3m,/r/wallstreetbets/comments/11ejs3m/what_are_yo...,3,Comment,2023-02-28 17:59:55
435003,imperator285,Sometimes I wish I could have the remote contr...,1677628795,jaf510w,11ejs3m,/r/wallstreetbets/comments/11ejs3m/what_are_yo...,1,Comment,2023-02-28 17:59:55
435004,Aarschotdachaubucha,Better the solid 7 who works their ass off at ...,1677628788,jaf50fu,11ejs3m,/r/wallstreetbets/comments/11ejs3m/what_are_yo...,2,Comment,2023-02-28 17:59:48
435006,sonofalando,Crammer,1677628782,jaf500h,11ejs3m,/r/wallstreetbets/comments/11ejs3m/what_are_yo...,1,Comment,2023-02-28 17:59:42
...,...,...,...,...,...,...,...,...,...
435093,InIce1234,Zoom out to see a Bear nightmare. imgemotet5_2...,1677628525,jaf4f1h,11ejs3m,/r/wallstreetbets/comments/11ejs3m/what_are_yo...,2,Comment,2023-02-28 17:55:25
435094,brook___trout,Why stonk go down,1677628508,jaf4dop,11ejs3m,/r/wallstreetbets/comments/11ejs3m/what_are_yo...,3,Comment,2023-02-28 17:55:08
435095,Pickle_Rick_Goat,What strike,1677628508,jaf4dmu,11ejs3m,/r/wallstreetbets/comments/11ejs3m/what_are_yo...,1,Comment,2023-02-28 17:55:08
435098,akrebo18,Squeeze deez nuggets,1677628496,jaf4clg,11ejs3m,/r/wallstreetbets/comments/11ejs3m/what_are_yo...,3,Comment,2023-02-28 17:54:56


In [28]:
data_cleaned[data_cleaned['id'] == '11ejs3m']

,type,id,title,upvote_ratio,author,created_utc,score,permalink,created_timestamp,title_body
26516,Post,11ejs3m,"What Are Your Moves Tomorrow, March 01, 2023",1.0,OPINION_IS_UNPOPULAR,1677618038,2,/r/wallstreetbets/comments/11ejs3m/what_are_yo...,2023-02-28 15:00:38,"What Are Your Moves Tomorrow, March 01, 2023 W..."


In [29]:
combined_data = data_cleaned.merge(comments_cleaned, how='left', left_on='id', right_on='link_id', suffixes=('_posts', '_comments'))
combined_data.head()

,type_posts,id_posts,title,upvote_ratio,author_posts,created_utc_posts,score_posts,permalink_posts,created_timestamp_posts,title_body,author_comments,body,created_utc_comments,id_comments,link_id,permalink_comments,score_comments,type_comments,created_timestamp_comments
0,Post,rtcytw,Possible reasons you arent 3D printing Tendy T...,0.0,[deleted],1641016911,1,/r/wallstreetbets/comments/rtcytw/possible_rea...,2022-01-01 00:01:51,Possible reasons you arent 3D printing Tendy T...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,Post,rtczjg,deleted by user,0.0,[deleted],1641016983,1,/r/wallstreetbets/comments/rtczjg/deleted_by_u...,2022-01-01 00:03:03,deleted by user removed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,Post,rtd4ey,Happy New Years you retards,0.0,carsonthecarsinogen,1641017504,1,/r/wallstreetbets/comments/rtd4ey/happy_new_ye...,2022-01-01 00:11:44,Happy New Years you retards nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,Post,rtd7ym,deleted by user,0.0,[deleted],1641017906,1,/r/wallstreetbets/comments/rtd7ym/deleted_by_u...,2022-01-01 00:18:26,deleted by user removed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,Post,rtdg3p,I DIDNT HEAR NO BELL,0.0,[deleted],1641018757,166,/r/wallstreetbets/comments/rtdg3p/i_didnt_hear...,2022-01-01 00:32:37,I DIDNT HEAR NO BELL deleted,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [30]:
combined_data.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 279174 entries, 0 to 279173
Data columns (total 19 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   type_posts                  279174 non-null  object        
 1   id_posts                    279174 non-null  object        
 2   title                       279174 non-null  object        
 3   upvote_ratio                279174 non-null  float64       
 4   author_posts                279174 non-null  object        
 5   created_utc_posts           279174 non-null  int64         
 6   score_posts                 279174 non-null  int64         
 7   permalink_posts             279174 non-null  object        
 8   created_timestamp_posts     279174 non-null  datetime64[ns]
 9   title_body                  279174 non-null  object        
 10  author_comments             37219 non-null   object        
 11  body                        37219 non-n

In [31]:
# data_cleaned = data_cleaned.drop(['upvote_ratio', 'author', 'created_utc', 'score', 'permalink'], axis=1)
data_cleaned.to_csv('Posts_Cleaned.csv', index=False)

In [32]:
# comments_cleaned = comments_cleaned.drop(['author', 'created_utc', 'score', 'permalink', 'created_timestamp'], axis=1)
comments_cleaned.to_csv('Comments_Cleaned.csv', index=False)

In [33]:
combined_data.to_csv('Combined_Data.csv', index=False)